In [ ]:
#Mount Google Drive to enable access to any directory on Drive inside the Colab notebook.
from google.colab import drive
drive.mount('/content/drive')

#Setting up PySpark in Colab, designate where the installer should be located
%cd /content/drive/MyDrive/Big_Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Big_Data


In [ ]:
#Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Spark Installer 
!wget -v https://dlcdn.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz 

--2022-11-27 22:37:28--  https://dlcdn.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231842529 (221M) [application/x-gzip]
Saving to: ‘spark-3.1.3-bin-hadoop3.2.tgz.1’

spark-3.1.3-bin-had 100%[===================>] 221.10M  38.3MB/s    in 5.6s    

2022-11-27 22:37:36 (39.7 MB/s) - ‘spark-3.1.3-bin-hadoop3.2.tgz.1’ saved [231842529/231842529]



In [ ]:
#Untar the Spark installer
!tar -xvf spark-3.1.3-bin-hadoop3.2.tgz

#Install findspark
!pip install -q findspark

#Set environment variables - Java and Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/MyDrive/Big_Data/spark-3.1.3-bin-hadoop3.2"

#Create local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark-3.1.3-bin-hadoop3.2/
spark-3.1.3-bin-hadoop3.2/bin/
spark-3.1.3-bin-hadoop3.2/bin/pyspark.cmd
spark-3.1.3-bin-hadoop3.2/bin/spark-submit
spark-3.1.3-bin-hadoop3.2/bin/spark-submit.cmd
spark-3.1.3-bin-hadoop3.2/bin/spark-class2.cmd
spark-3.1.3-bin-hadoop3.2/bin/spark-shell2.cmd
spark-3.1.3-bin-hadoop3.2/bin/pyspark2.cmd
spark-3.1.3-bin-hadoop3.2/bin/docker-image-tool.sh
spark-3.1.3-bin-hadoop3.2/bin/run-example.cmd
spark-3.1.3-bin-hadoop3.2/bin/spark-submit2.cmd
spark-3.1.3-bin-hadoop3.2/bin/beeline.cmd
spark-3.1.3-bin-hadoop3.2/bin/beeline
spark-3.1.3-bin-hadoop3.2/bin/spark-shell
spark-3.1.3-bin-hadoop3.2/bin/find-spark-home
spark-3.1.3-bin-hadoop3.2/bin/sparkR2.cmd
spark-3.1.3-bin-hadoop3.2/bin/find-spark-home.cmd
spark-3.1.3-bin-hadoop3.2/bin/sparkR
spark-3.1.3-bin-hadoop3.2/bin/spark-class
spark-3.1.3-bin-hadoop3.2/bin/spark-sql2.cmd
spark-3.1.3-bin-hadoop3.2/bin/load-spark-env.cmd
spark-3.1.3-bin-hadoop3.2/bin/run-example
spark-3.1.3-bin-hadoop3.2/bin/spark-sql
spark-3.1.3-b

In [ ]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

ValueError: ignored

## Loading tweets dataset

In [ ]:
# Read the json twitter data

Twdf1=pd.read_json('/content/drive/MyDrive/Big_Data/doge_tweets_aug2021.json',lines = True) 

In [ ]:
#get only the content and date columns from the twitter dataset
TwDF = Twdf1
TwDF = TwDF.filter(['date', 'content'], axis=1)

TwDF['date'] = pd.to_datetime(TwDF['date'])
TwDF['date'] = TwDF['date'].dt.floor('h')
TwDF

,date,content
0,2021-08-01 23:00:00+00:00,The current value of 1 DOGE in USD is: $0.2045...
1,2021-08-01 23:00:00+00:00,Burger King Starts Accepting Dogecoin in Brazi...
2,2021-08-01 23:00:00+00:00,$DOGE #Dogecoin has been moving up since I sai...
3,2021-08-01 23:00:00+00:00,50% Off!\n\nOlympia USA Hopkins 18 in. Navy Ba...
4,2021-08-01 23:00:00+00:00,@Tokyo_Doge @FabriLemus7 @dogecoin @elonmusk L...
...,...,...
76961,2021-08-07 00:00:00+00:00,@BabyDogeCoin @elonmusk @MARCIANOPHONE @Alex_H...
76962,2021-08-07 00:00:00+00:00,Bitcoin Price (USD): 42846.08 \nEthereum Price...
76963,2021-08-07 00:00:00+00:00,🚨BREAKING-NEWS🚨\n\n🗞️ Binance US CEO Steps Dow...
76964,2021-08-07 00:00:00+00:00,The current price of one Baby Doge Coin is: $0...


In [ ]:
#Load the ethereum data and filter out only the date and close columns
ETHDF=pd.read_csv('/content/drive/MyDrive/Big_Data/DogeVsUsd_Aug2021.csv',error_bad_lines=False,engine = 'python', header = None, parse_dates=True) 

ETHDF = ETHDF.filter([0, 5])
ETHDF = ETHDF[1:]
ETHDF[0] = pd.to_datetime(ETHDF[0])
ETHDF

,0,5
1,2021-08-31 23:00:00,0.27878
2,2021-08-31 22:00:00,0.27633
3,2021-08-31 21:00:00,0.27703
4,2021-08-31 20:00:00,0.27708
5,2021-08-31 19:00:00,0.2792
...,...,...
740,2021-08-01 04:00:00,0.2143
741,2021-08-01 03:00:00,0.21671
742,2021-08-01 02:00:00,0.21305
743,2021-08-01 01:00:00,0.21045


In [ ]:
#Creating a sql dataframe
FullDataTw=sql.createDataFrame(TwDF.astype(str))
FullDataEth=sql.createDataFrame(ETHDF) #creating pandas df and then changing it to pyspark df

In [ ]:
FullDataTw = FullDataTw.dropna() #getting rid of full empty rows
FullDataEth.head(5)

[Row(0=datetime.datetime(2021, 8, 31, 23, 0), 5='0.27878'),
 Row(0=datetime.datetime(2021, 8, 31, 22, 0), 5='0.27633'),
 Row(0=datetime.datetime(2021, 8, 31, 21, 0), 5='0.27703'),
 Row(0=datetime.datetime(2021, 8, 31, 20, 0), 5='0.27708'),
 Row(0=datetime.datetime(2021, 8, 31, 19, 0), 5='0.2792')]

In [ ]:
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
FullDataTw = FullDataTw.withColumnRenamed('date', 'DateTime') #setting column names of Twitter dataset
FullDataTw = FullDataTw.withColumnRenamed('content', 'Tweet')
FullDataEth = FullDataEth.withColumnRenamed('0', 'DateTime') #setting column names of Ethereum price dataset
FullDataEth = FullDataEth.withColumnRenamed('5', 'Price')

In [ ]:
FullDataTw.head(5)

[Row(DateTime='2021-08-01 23:00:00+00:00', Tweet='The current value of 1 DOGE in USD is: $0.20455 (📉 down 0.008550 so far today). #dogecoin'),
 Row(DateTime='2021-08-01 23:00:00+00:00', Tweet='Burger King Starts Accepting Dogecoin in Brazil — but Only for Dog\xa0Treats https://t.co/IjkgmYjsHB'),
 Row(DateTime='2021-08-01 23:00:00+00:00', Tweet='$DOGE #Dogecoin has been moving up since I said it started being bullish and everyone laughed at me lol. https://t.co/579X7Awjg1'),
 Row(DateTime='2021-08-01 23:00:00+00:00', Tweet='50% Off!\n\nOlympia USA Hopkins 18 in. Navy Backpack, Blue\n\nhttps://t.co/Ty8T58eLlU\n\n#BwcDeals #Dogecoin #clearthelists #dailydeals  #DealsAndSteals https://t.co/3IjiDnhiXq'),
 Row(DateTime='2021-08-01 23:00:00+00:00', Tweet='@Tokyo_Doge @FabriLemus7 @dogecoin @elonmusk Lezgooow #tokyodoge #teamtokyodoge')]

In [ ]:
FullDataEth.head(5)

[Row(DateTime=datetime.datetime(2021, 8, 31, 23, 0), Price='0.27878'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 22, 0), Price='0.27633'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 21, 0), Price='0.27703'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 20, 0), Price='0.27708'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 19, 0), Price='0.2792')]

In [ ]:
FullDataEth.show()

+-------------------+-------+
|           DateTime|  Price|
+-------------------+-------+
|2021-08-31 23:00:00|0.27878|
|2021-08-31 22:00:00|0.27633|
|2021-08-31 21:00:00|0.27703|
|2021-08-31 20:00:00|0.27708|
|2021-08-31 19:00:00| 0.2792|
|2021-08-31 18:00:00| 0.2792|
|2021-08-31 17:00:00|0.27747|
|2021-08-31 16:00:00| 0.2801|
|2021-08-31 15:00:00|0.27868|
|2021-08-31 14:00:00|0.27543|
|2021-08-31 13:00:00|0.27964|
|2021-08-31 12:00:00|0.27727|
|2021-08-31 11:00:00|0.27915|
|2021-08-31 10:00:00|0.27551|
|2021-08-31 09:00:00|0.27498|
|2021-08-31 08:00:00| 0.2759|
|2021-08-31 07:00:00|0.27627|
|2021-08-31 06:00:00|0.27103|
|2021-08-31 05:00:00|0.27327|
|2021-08-31 04:00:00|0.27354|
+-------------------+-------+
only showing top 20 rows



## Pre-Processing Twitter dataframe

In [ ]:
Tw_samp = FullDataTw  #copy of data

In [ ]:
!pip install -q tweet-preprocessor

In [ ]:
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
def function_udf(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['Tweet']))
CleanDF.show(3)

+--------------------+--------------------+--------------------+
|            DateTime|               Tweet|       CleanedTweets|
+--------------------+--------------------+--------------------+
|2021-08-01 23:00:...|The current value...|The current value...|
|2021-08-01 23:00:...|Burger King Start...|Burger King Start...|
|2021-08-01 23:00:...|$DOGE #Dogecoin h...|DOGE Dogecoin has...|
+--------------------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Sentiment analysis using Vader packages

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # using vader sentiment to generate the sentiment analysis on twitter tweets
analyser = SentimentIntensityAnalyzer()
def senti_score_udf(sentence):
    snt = analyser.polarity_scores(sentence)
    return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['CleanedTweets'])[0])
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['CleanedTweets'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['CleanedTweets'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['CleanedTweets'])[3])
CleanDF.show(3)

+--------------------+--------------------+--------------------+-----+-----+-----+------+
|            DateTime|               Tweet|       CleanedTweets|p_neg|p_neu|p_pos|p_comp|
+--------------------+--------------------+--------------------+-----+-----+-----+------+
|2021-08-01 23:00:...|The current value...|The current value...|  0.0|0.876|0.124|  0.34|
|2021-08-01 23:00:...|Burger King Start...|Burger King Start...|  0.0|0.847|0.153|0.2023|
|2021-08-01 23:00:...|$DOGE #Dogecoin h...|DOGE Dogecoin has...|  0.0|0.746|0.254|0.7003|
+--------------------+--------------------+--------------------+-----+-----+-----+------+
only showing top 3 rows



In [ ]:
#changing datatype
CleanDF = CleanDF.withColumn("DateTime",CleanDF['DateTime'].cast(TimestampType()))
CleanDF.show(3)

+-------------------+--------------------+--------------------+-----+-----+-----+------+
|           DateTime|               Tweet|       CleanedTweets|p_neg|p_neu|p_pos|p_comp|
+-------------------+--------------------+--------------------+-----+-----+-----+------+
|2021-08-01 23:00:00|The current value...|The current value...|  0.0|0.876|0.124|  0.34|
|2021-08-01 23:00:00|Burger King Start...|Burger King Start...|  0.0|0.847|0.153|0.2023|
|2021-08-01 23:00:00|$DOGE #Dogecoin h...|DOGE Dogecoin has...|  0.0|0.746|0.254|0.7003|
+-------------------+--------------------+--------------------+-----+-----+-----+------+
only showing top 3 rows



In [ ]:
FinalTw = CleanDF.selectExpr("DateTime as Date_Time", "CleanedTweets as Cleaned_Tweets", "p_neg","p_neu","p_pos","p_comp")
FinalTw.show(5) #selecting necessary columns

+-------------------+--------------------+-----+-----+-----+------+
|          Date_Time|      Cleaned_Tweets|p_neg|p_neu|p_pos|p_comp|
+-------------------+--------------------+-----+-----+-----+------+
|2021-08-01 23:00:00|The current value...|  0.0|0.876|0.124|  0.34|
|2021-08-01 23:00:00|Burger King Start...|  0.0|0.847|0.153|0.2023|
|2021-08-01 23:00:00|DOGE Dogecoin has...|  0.0|0.746|0.254|0.7003|
|2021-08-01 23:00:00|50 Off Olympia US...|  0.0|  1.0|  0.0|   0.0|
|2021-08-01 23:00:00|Lezgooow tokyodog...|  0.0|  1.0|  0.0|   0.0|
+-------------------+--------------------+-----+-----+-----+------+
only showing top 5 rows



In [ ]:
from datetime import datetime 
from dateutil import parser

In [ ]:

FinalEth = FullDataEth
FinalEth.show(5)#In this cell, casting to timesstamp, changing col names and casting price type to double

+-------------------+-------+
|           DateTime|  Price|
+-------------------+-------+
|2021-08-31 23:00:00|0.27878|
|2021-08-31 22:00:00|0.27633|
|2021-08-31 21:00:00|0.27703|
|2021-08-31 20:00:00|0.27708|
|2021-08-31 19:00:00| 0.2792|
+-------------------+-------+
only showing top 5 rows



## Dataframes Look like this...

In [ ]:
FinalTw.printSchema()

root
 |-- Date_Time: timestamp (nullable = true)
 |-- Cleaned_Tweets: string (nullable = true)
 |-- p_neg: float (nullable = true)
 |-- p_neu: float (nullable = true)
 |-- p_pos: float (nullable = true)
 |-- p_comp: float (nullable = true)



In [ ]:
FinalEth.printSchema()
FinalEth.count()

root
 |-- DateTime: timestamp (nullable = true)
 |-- Price: string (nullable = true)



744

## Truncating timestamps to hours and then grouping them by hour

In [ ]:
dt_truncated = ((round(unix_timestamp(col('Date_Time')) / 3600) * 3600).cast('timestamp'))
FinalTw = FinalTw.withColumn('dt_truncated', dt_truncated)
FinalTw = FinalTw.selectExpr("dt_truncated as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
UTC = ((unix_timestamp(col('Date_Time'))+ 5*60*60).cast('timestamp'))
FinalTw = FinalTw.withColumn('UTC', UTC)
FinalTw = FinalTw.selectExpr("UTC as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
FinalTw.show()

+-------------------+--------------------+-----+-----+-----+-------+
|          Date_Time|      Cleaned_Tweets|p_neg|p_neu|p_pos| p_comp|
+-------------------+--------------------+-----+-----+-----+-------+
|2021-08-02 04:00:00|The current value...|  0.0|0.876|0.124|   0.34|
|2021-08-02 04:00:00|Burger King Start...|  0.0|0.847|0.153| 0.2023|
|2021-08-02 04:00:00|DOGE Dogecoin has...|  0.0|0.746|0.254| 0.7003|
|2021-08-02 04:00:00|50 Off Olympia US...|  0.0|  1.0|  0.0|    0.0|
|2021-08-02 04:00:00|Lezgooow tokyodog...|  0.0|  1.0|  0.0|    0.0|
|2021-08-02 04:00:00|Sending 100 Dogec...|  0.0|  1.0|  0.0|    0.0|
|2021-08-02 04:00:00|This dude was hyp...|0.086|0.839|0.075| -0.099|
|2021-08-02 04:00:00|Destroyer of Shor...|0.231|0.769|  0.0|-0.4588|
|2021-08-02 04:00:00|giftsforher bitco...|  0.0|0.884|0.116| 0.4404|
|2021-08-02 04:00:00|tumblr twitter fa...|  0.0|0.827|0.173| 0.5574|
|2021-08-02 04:00:00|business business...|  0.0|  1.0|  0.0|    0.0|
|2021-08-02 04:00:00|linkedin twit

In [ ]:
FinalTw.registerTempTable("temp")
FinalTw_avg = sql.sql("SELECT Date_Time As DateTime,AVG(p_neg) as P_Neg,AVG(p_neu) as P_Neu,AVG(p_pos) as P_Pos,AVG(p_comp) as P_Comp FROM temp GROUP BY Date_Time")
FinalTw_avg.show(1)

+-------------------+-------------------+------------------+-------------------+-------------------+
|           DateTime|              P_Neg|             P_Neu|              P_Pos|             P_Comp|
+-------------------+-------------------+------------------+-------------------+-------------------+
|2021-08-10 04:00:00|0.02295022629774534|0.8492013599673008|0.11878732993473969|0.22493167258630511|
+-------------------+-------------------+------------------+-------------------+-------------------+
only showing top 1 row



In [ ]:
#This cell is just to collect all the corpus per hour(for the future work)
from pyspark.sql import functions as f
df_with_text = FinalTw.groupby("Date_Time").agg(f.concat_ws(" ", f.collect_list(FinalTw.Cleaned_Tweets)))
df_with_text.show(1)

+-------------------+------------------------------------------+
|          Date_Time|concat_ws( , collect_list(Cleaned_Tweets))|
+-------------------+------------------------------------------+
|2021-08-10 04:00:00|                      The current value...|
+-------------------+------------------------------------------+
only showing top 1 row



In [ ]:
FinalTw_avg.head(5)

[Row(DateTime=datetime.datetime(2021, 8, 10, 4, 0), P_Neg=0.02295022629774534, P_Neu=0.8492013599673008, P_Pos=0.11878732993473969, P_Comp=0.22493167258630511),
 Row(DateTime=datetime.datetime(2021, 8, 3, 2, 0), P_Neg=0.017873015999794007, P_Neu=0.8612571457075694, P_Pos=0.10182539661015783, P_Comp=0.21920349102408168),
 Row(DateTime=datetime.datetime(2021, 8, 5, 20, 0), P_Neg=0.02057790377364618, P_Neu=0.8676968865286527, P_Pos=0.11170821464897553, P_Comp=0.2337271936195599),
 Row(DateTime=datetime.datetime(2021, 8, 7, 14, 0), P_Neg=0.015428270028613288, P_Neu=0.8833291158394472, P_Pos=0.09913291107030106, P_Comp=0.18248755140583725),
 Row(DateTime=datetime.datetime(2021, 8, 3, 16, 0), P_Neg=0.01714657990911108, P_Neu=0.8412736182977788, P_Pos=0.0959869697568665, P_Comp=0.20514625166497324)]

In [ ]:
FinalEth.head(5)

[Row(DateTime=datetime.datetime(2021, 8, 31, 23, 0), Price='0.27878'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 22, 0), Price='0.27633'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 21, 0), Price='0.27703'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 20, 0), Price='0.27708'),
 Row(DateTime=datetime.datetime(2021, 8, 31, 19, 0), Price='0.2792')]

In [ ]:
FinalTw_avg.registerTempTable("avgs")
FinalEth.registerTempTable("prices")
results = sql.sql("SELECT avgs.DateTime, avg(P_Comp) as P_Comp, avg(Price) as Price FROM avgs JOIN prices ON avgs.DateTime = prices.DateTime group by avgs.DateTime order by avgs.DateTime")
results=results.withColumn("P_Comp", lit(results.agg({"P_Comp": "max"}).collect()[0][0]) - results.P_Comp)
results.show(10)

+-------------------+-------------------+-------+
|           DateTime|             P_Comp|  Price|
+-------------------+-------------------+-------+
|2021-08-01 05:00:00|0.22145436779941766|0.21343|
|2021-08-01 06:00:00|0.15851342901618196|0.21316|
|2021-08-01 07:00:00|0.22179061601852285|0.21247|
|2021-08-01 08:00:00|0.17863262226300708|0.21289|
|2021-08-01 09:00:00|0.18514932244814186|0.21389|
|2021-08-01 10:00:00|0.24531455628008028|0.21134|
|2021-08-01 11:00:00| 0.2752749987677494|0.21136|
|2021-08-01 12:00:00| 0.2056156067355048|0.21147|
|2021-08-01 13:00:00|0.22150343403066555| 0.2105|
|2021-08-01 14:00:00| 0.2512646679080532|0.21019|
+-------------------+-------------------+-------+
only showing top 10 rows



In [ ]:
results.show()

+-------------------+-------------------+-------+
|           DateTime|             P_Comp|  Price|
+-------------------+-------------------+-------+
|2021-08-01 05:00:00|0.10220562739883218|0.21343|
|2021-08-01 06:00:00|0.16514656618206788|0.21316|
|2021-08-01 07:00:00|  0.101869379179727|0.21247|
|2021-08-01 08:00:00|0.14502737293524276|0.21289|
|2021-08-01 09:00:00|0.13851067275010798|0.21389|
|2021-08-01 10:00:00|0.07834543891816956|0.21134|
|2021-08-01 11:00:00|0.04838499643050043|0.21136|
|2021-08-01 12:00:00|0.11804438846274504|0.21147|
|2021-08-01 13:00:00|0.10215656116758429| 0.2105|
|2021-08-01 14:00:00|0.07239532729019665|0.21019|
|2021-08-01 15:00:00|0.09566502714165925|0.21051|
|2021-08-01 16:00:00|0.09724580107333158|0.20975|
|2021-08-01 17:00:00| 0.1441642690752434|0.20877|
|2021-08-01 18:00:00|0.10529089246894113| 0.2094|
|2021-08-01 19:00:00|0.08950401681164902|0.21073|
|2021-08-01 20:00:00|0.07025749747175725|0.21104|
|2021-08-01 21:00:00|0.12462506440030116|0.21033|


In [ ]:
#use this csv as the input to the "ML Model Prediction" file
results.repartition(1).write.csv("/content/drive/MyDrive/Big_Data/Final.csv") #this will write df to single csv instead of writing diff csv acc to partitions 